In [1]:
import pandas as pd
import numpy as np
import json
from tqdm.notebook import tqdm
from functools import partial

import multiprocessing
from multiprocessing import Process, Manager

import warnings

warnings.filterwarnings(action='ignore')

data_dir = './Data/'

In [2]:
song_meta_df = pd.read_json(data_dir + 'song_meta_data_v3.json')
song_meta_df = song_meta_df.sort_values('id')
song_meta_df = song_meta_df.reset_index(drop = True)
song_meta_df['song_embedding_idx'] = song_meta_df.index

song_id = song_meta_df['id'].tolist()
song_embedding_idx = song_meta_df['song_embedding_idx'].tolist()

song_id2song_embedding_idx = {}
song_embedding_idx2song_id = {}
for id, idx in zip(song_id, song_embedding_idx):
    song_id2song_embedding_idx[id] = idx
    song_embedding_idx2song_id[idx] = id

word2vec_embedding = np.load(data_dir + 'word2vec_embedding.npy')
mel_embedding = np.load(data_dir + 'tanh_mel_embeding_loss55.npy')
cos_embedding = np.load(data_dir + 'get_cos_embedding_63.npy')

In [ ]:
print(multiprocessing.cpu_count())

In [ ]:
from make_ranking_data_multiprocessig import get_json_data

song_id_li = song_meta_df['id'].tolist()

manager = Manager()

song_id2playlist = manager.dict()

pool = multiprocessing.Pool(processes = 7)

pool.map(partial(get_json_data,
                 word2vec_embedding = word2vec_embedding,
                 mel_embedding = mel_embedding,
                 cos_embedding = cos_embedding,
                 song_id2song_embedding_idx = song_id2song_embedding_idx,
                 song_embedding_idx2song_id = song_embedding_idx2song_id,
                 song_id2playlist = song_id2playlist),
         tqdm(song_id_li))

pool.close()
pool.join()

In [ ]:
song_id2playlist = dict(song_id2playlist)
len(song_id2playlist)

In [6]:
with open(data_dir + 'song_id2playlist_wmc.json', 'w') as f:
    json.dump(song_id2playlist, f)

In [7]:
with open(data_dir + 'song_id2playlist_wmc.json', 'r') as f:
    new_data = json.load(f)